# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3 as sq
import re
import pickle

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stopwords_english = stopwords.words('english')

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix


[nltk_data] Downloading package punkt to /Users/julio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/julio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages_df', con=engine, schema='main', index_col='id')
df.head()

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df['message'].to_frame().head(n=1)

,message
id,
2,Weather update - a cold front from Cuba that c...


In [4]:
df.loc[:,"related":].head(n=1)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
y = df.loc[:,"related":]
y.head(n=1)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# X = df[['message', 'genre']]
X = df['message']
X.head()

id
2     Weather update - a cold front from Cuba that c...
7               Is the Hurricane over or is it not over
8                       Looking for someone but no name
9     UN reports Leogane 80-90 destroyed. Only Hospi...
12    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [24]:
X.iloc[12:20].to_list()

['I am in Thomassin number 32, in the area named Pyron. I would like to have some water. Thank God we are fine, but we desperately need water. Thanks',
 "Let's do it together, need food in Delma 75, in didine area",
 'More information on the 4636 number in order for me to participate. ( To see if I can use it )',
 'A Comitee in Delmas 19, Rue ( street ) Janvier, Impasse Charite #2. We have about 500 people in a temporary shelter and we are in dire need of Water, Food, Medications, Tents and Clothes. Please stop by and see us.',
 'We need food and water in Klecin 12. We are dying of hunger. Impasse Chretien Klecin 12 extended ( extension ) We are hungry and sick.',
 'are you going to call me or do you want me to call ou? let me know?',
 "I don't understand how to use this thing 4636.",
 'I would like to know if the earthquake is over. Thanks']

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = tok.strip()
        if clean_tok in stopwords_english:
            continue
        else:
            lemma = lemmatizer.lemmatize(clean_tok, pos='v')
            clean_tokens.append(lemma)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
# column_trans = ColumnTransformer([
#     ('genre_category', OneHotEncoder(dtype='int'),['genre']), 
#     ('message_tf', TfidfVectorizer(tokenizer=tokenize), 'message')], 
#     remainder='drop')

# pipeline = Pipeline([
#     ('transformers', column_trans),
#     ('multi_clf', MultiOutputClassifier(RandomForestClassifier()))
# ])

# param_grid = {
#     'transformers__message_tf__tokenizer': [tokenize],
#     'transformers__message_tf__max_df': [1.0, 0.95],
#     'transformers__message_tf__use_idf': [True, False],
#     'multi_clf__estimator__n_jobs': [-1],    
#     'multi_clf__estimator__n_estimators': [10, 20, 100],
#     'multi_clf__n_jobs': [-1]
# }

In [9]:
pipeline = Pipeline([
    ('tf', TfidfVectorizer()),
    ('multi_clf', MultiOutputClassifier(RandomForestClassifier())),
])

parameters = {
    'tf__tokenizer': [tokenize],
    'tf__max_df': [1.0, 0.95],
    'tf__use_idf': [True, False],
    'multi_clf__estimator__n_jobs': [-1],    
    'multi_clf__estimator__n_estimators': [10, 20, 100]
}

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)

In [11]:
len(X_train) == len(y_train)

True

In [12]:
y_train.shape

(13107, 36)

In [13]:
X_train.shape

(13107,)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
search = GridSearchCV(pipeline, param_grid=parameters)
model = search.fit(X_train, y_train)

In [15]:
y_pred = model.predict(X_test)

In [16]:
for i, col in enumerate(y.columns):
    col_preds = y_pred[:,i]
    col_test = y_test[col]
    print(col)
    print(classification_report(col_test, col_preds))    

related
              precision    recall  f1-score   support

           0       0.68      0.39      0.50      3032
           1       0.83      0.94      0.89      9979
           2       0.42      0.34      0.38        97

    accuracy                           0.81     13108
   macro avg       0.64      0.56      0.59     13108
weighted avg       0.80      0.81      0.79     13108

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     10835
           1       0.83      0.51      0.63      2273

    accuracy                           0.90     13108
   macro avg       0.87      0.74      0.78     13108
weighted avg       0.89      0.90      0.89     13108

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13043
           1       0.00      0.00      0.00        65

    accuracy                           1.00     13108
   macro avg       0.50      0.50      0.50     1310

/usr/local/anaconda3/envs/py38_market/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98     12250
           1       0.89      0.34      0.49       858

    accuracy                           0.95     13108
   macro avg       0.92      0.67      0.73     13108
weighted avg       0.95      0.95      0.94     13108

food
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     11631
           1       0.83      0.58      0.69      1477

    accuracy                           0.94     13108
   macro avg       0.89      0.78      0.83     13108
weighted avg       0.94      0.94      0.93     13108

shelter
              precision    recall  f1-score   support

           0       0.94      0.99      0.96     11929
           1       0.82      0.34      0.48      1179

    accuracy                           0.93     13108
   macro avg       0.88      0.67      0.72     13108
weighted avg       0.93      0.93      0.92     13108

clothi

In [17]:
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.261
Best parameters set:
	multi_clf__estimator__n_estimators: 100
	multi_clf__estimator__n_jobs: -1
	tf__max_df: 0.95
	tf__tokenizer: <function tokenize at 0x1265c6dc0>
	tf__use_idf: True


### 9. Export your model as a pickle file

In [18]:
file_name = 'model.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(model, f)

In [19]:
with open(file_name, 'rb') as f:
    check_model = pickle.load(f)

In [20]:
y_pred = check_model.predict(X_test)

In [21]:
check_model.best_params_

{'multi_clf__estimator__n_estimators': 100,
 'multi_clf__estimator__n_jobs': -1,
 'tf__max_df': 0.95,
 'tf__tokenizer': <function __main__.tokenize(text)>,
 'tf__use_idf': True}

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.